In [1]:
import numpy as np
import geopandas as gpd
import pandas as pd
import codecs
import requests
#import rasterio as rio 
#from rasterstats import zonal_stats, point_query
import xarray as xr


In [2]:
#########################################################################
############################ USER INPUTS ################################
#########################################################################

# DOMAIN
# choose the modeling domain
domain = 'YUKO'

# PATHS
#path to CSO domain
domains_resp = requests.get("https://raw.githubusercontent.com/NPRB/02_preprocess_python/main/NPRB_domains.json")
domains = domains_resp.json()
# CSO projection
mod_proj = domains[domain]['mod_proj']

# DATES
#start calibration date    
st_dt = '2011-10-01'
#end calibration date
ed_dt = '2018-09-30'
#########################################################################

In [3]:
def get_paths(domain):
    # path to domain data folder 
    dataPath = '/nfs/attic/dfh/2020_NPRB/data/SMinputs/'+domain+'/'

    # path to station gdf
    gdfpath = '/nfs/attic/dfh/2020_NPRB/data/snotel/'+domain+'/Station_sites_SWEDmeters_'+domain+'_'+st_dt+'_'+ed_dt+'.geojson'

    # path to DEM ascii
    DEMpath = dataPath+domain+'_dem.asc'

    #path to VEG .asc
    VEGpath = dataPath+domain+'_veg.asc'

    #path to lat .asc
    LATpath = dataPath+domain+'_grid_lat.asc'

    #path to lon .asc
    LONpath = dataPath+domain+'_grid_lon.asc'

    # VEG outfile path
    outVEGpath = dataPath+domain+'_veg_line.asc'

    # DEM outfile path
    outDEMpath = dataPath+domain+'_dem_line.asc'

    # Line outfile path
    outFpath = dataPath+'/snowmodel_line_pts.dat'

    #lon outfile path
    outLONpath = dataPath+domain+'_grid_lon_line.asc'

    #lat outfile path
    outLATpath = dataPath+domain+'_grid_lat_line.asc'

    return(dataPath,gdfpath,DEMpath,VEGpath,LATpath,LONpath,outFpath,outVEGpath,outDEMpath,outLONpath,outLATpath)

In [4]:
def snowmodel_line_input(rasterpath,gdfpath,modproj,outpath):
    # open raster
    ds = xr.open_rasterio(rasterpath)
    # open station geodataframe
    stngdf = gpd.read_file(gdfpath)
    #project stations to match model
    stnproj = stngdf.to_crs(mod_proj)
    # get count of number of stations
    count = 1
    f= open(outFpath,"w+")
    for i in range(stngdf.shape[0]):
        # x index
        # add 1 since fortran indexing starts with 1, not 0
        xidx = ds.indexes["x"].get_loc(stnproj.iloc[i].geometry.x,  method="nearest")+1
        # x easting
        xval = int(ds.x.sel(x=stnproj.iloc[i].geometry.x,  method="nearest").values)
        # y index
        yidx_r = ds.indexes["y"].get_loc(stnproj.iloc[i].geometry.y,  method="nearest")
        # need to convert this to indexing starting at the lower right corner for SM
        # since ascii files give you indexing starting in the upper right corner
        yidx = len(ds.y)-yidx_r
        # y northing
        yval =  int(ds.y.sel(y=stnproj.iloc[i].geometry.y,  method="nearest").values)
        #Print out .dat file 
        f.write('{:08.0f}\t'.format(count)+'{:10.0f}\t'.format(xidx)+'{:10.0f}\t'.format(yidx)+
                '{:10.0f}\t'.format(xval)+'{:10.0f}\t\n'.format(yval))
        count=count+1
    f.close() 

In [5]:
# function to extract .asc files at SNOTEL locations
def field2line(rasterpath,gdfpath,modproj,outpath):
    # open raster
    ds = xr.open_rasterio(rasterpath)
    # open station geodataframe
    stngdf = gpd.read_file(gdfpath)
    #project stations to match model
    stnproj = stngdf.to_crs(mod_proj)

    data_line=[]
    for i in range(stngdf.shape[0]):
        info = ds.sel(x=stnproj.iloc[i].geometry.x, y=stnproj.iloc[i].geometry.y, method="nearest")
        data_line.append(str(info.values[0]))
    lines = open(rasterpath, 'r').readlines()
    head = 'ncols\t1\nnrows\t'+str(stnproj.shape[0])+'\n'+lines[2]+lines[3]+lines[4]+lines[5]
    data = ''
    for s in data_line:
        data += s +'\n'
    f= open(outpath,"w+")
    f.write(head+data)
    f.close()    

In [6]:
domain_list = [domain]

for domain in domain_list:
    dataPath,gdfpath,DEMpath,VEGpath,LATpath,LONpath,outFpath,outVEGpath,outDEMpath,outLONpath,outLATpath = get_paths(domain)

    # build snowmodel line file
    snowmodel_line_input(DEMpath,gdfpath,mod_proj,outFpath)

    #extract DEM
    field2line(DEMpath,gdfpath,mod_proj,outDEMpath)

    #extract VEG
    field2line(VEGpath,gdfpath,mod_proj,outVEGpath)

    #extract LAT
    field2line(LATpath,gdfpath,mod_proj,outLATpath)

    #extract LON
    field2line(LONpath,gdfpath,mod_proj,outLONpath)